# Document Analysis

### The "Bag-o-words" Approach

Creates a representation of the document, usually with a stop list.

A stop list is a list of words like "the", "and", or "but" that punctuate a document.

Possible methods:
- Term frequency (raw number of times a word appears)
- Normalized term frequency (percentage of times a word appears)
- Inverse document frequency (what percentage of documents does this word occur in?)

The combination of these methods is called TF-IDF.

TF-IDF gives a list of words that acts as a *frequency vector*, which we can compare with a Perceptron or SVM.

In [1]:
%matplotlib inline
import numpy as np
import random
import pandas as pd
import seaborn as sbn
import matplotlib.pyplot as plt

from math import log, sqrt
from sklearn import datasets
from sklearn.svm import SVC
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

In [2]:
doc = datasets.fetch_20newsgroups()

In [3]:
print(doc.keys())
print(doc.target_names)
print(doc.target)

dict_keys(['target', 'filenames', 'target_names', 'DESCR', 'data'])
['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
[7 4 4 ..., 3 1 8]


In [4]:
print(doc.target_names[doc.target[1]], "\n\n", doc.data[1])

comp.sys.mac.hardware 

 From: guykuo@carson.u.washington.edu (Guy Kuo)
Subject: SI Clock Poll - Final Call
Summary: Final call for SI clock reports
Keywords: SI,acceleration,clock,upgrade
Article-I.D.: shelley.1qvfo9INNc3s
Organization: University of Washington
Lines: 11
NNTP-Posting-Host: carson.u.washington.edu

A fair number of brave souls who upgraded their SI clock oscillator have
shared their experiences for this poll. Please send a brief message detailing
your experiences with the procedure. Top speed attained, CPU rated speed,
add on cards and adapters, heat sinks, hour of usage per day, floppy disk
functionality with 800 and 1.4 m floppies are especially requested.

I will be summarizing in the next two days, so please add to the network
knowledge base if you have done the clock upgrade and haven't answered this
poll. Thanks.

Guy Kuo <guykuo@u.washington.edu>



Basic algorithm:
- Manipulate text
- Get dictionary of words and their idf values
- Compare with cos(theta), return how similar they are in terms of the dot product

In [5]:
class DocumentAnalyzer:
    # Constructor manipulates documents into an easier-to-analyze format
    def __init__(self, documents):
        self.ALPHANUMERIC = "a b c d e f g h i j k l m n o p q r s t u v w x y z 0 1 2 3 4 5 6 7 8 9".split(' ')
        self.doclist = []
        self.termlist = {}
        for doc in documents:
            self.doclist.append(self.fix_text(doc))            
    
    # Splits a list into its constituent parts, removing all punctuation and numbers
    # Perhaps unfortunately, this also splits email addresses.
    def fix_text(self, text):
        new_text = ""
        for char in text.lower():
            if char in self.ALPHANUMERIC:
                new_text += char
            else:
                new_text += ' '
        return [x for x in new_text.split() if not x.isdigit() and not x in ENGLISH_STOP_WORDS]
    
    # Gets the term frequency of a word in a document
    # Uses Wikipedia's "log normalization" weighting scheme
    def _tf(self, doc_id, w):
        f = 0
        for word in self.doclist[doc_id]:
            if w == word:
                f += 1
        return log(1 + f)

    # Gets the inverse document frequency of a word in a collection of documents
    # Uses Wikipedia's "inverse frequency smooth" weighting scheme
    def _idf(self, t):
        n = 0
        for doc in self.doclist:
            if t in doc:
                n += 1
        # n should always be greater than 0, but just in case . . .
        if n > 0:
            return log(1 + len(self.doclist) / n)
        else:
            return 0

    # Gets the TF-IDF value of a word
    def _tf_idf(self, doc_id, term):
        return self._tf(doc_id, term) * self._idf(term)
    
    # Returns cos(theta) for two documents with the given ids
    def compare(self, a_id, b_id):
        # Lazy-load TF-IDF vectors for a and b
        if not a_id in self.termlist.keys():
            self.termlist[a_id] = {}
            for word in self.doclist[a_id]:
                self.termlist[a_id][word] = self._tf_idf(a_id, word)
        if b_id not in self.termlist.keys():
            self.termlist[b_id] = {}
            for word in self.doclist[b_id]:
                self.termlist[b_id][word] = self._tf_idf(b_id, word)
        
        # The dot product is defined as the sum of the elements in an elementwise-product of a and b.
        # It is also defined as the product of the magnitudes of a and b and cos(theta) where theta 
        # is the angle between a and b. So cos(theta) is the dot product dived by the product of the
        # magnitudes of a and b.
        dot_product = self.dot(self.termlist[a_id], self.termlist[b_id])
        a_mag, b_mag = self.magnitude(self.termlist[a_id]), self.magnitude(self.termlist[b_id])
        try:
            cos = dot_product / (a_mag * b_mag)
        except:
            cos = 0
            
        return cos
    
    # Takes two dictionaries of TF-IDF values and returns dot product
    def dot(self, a, b):
        total = 0
        for key in a.keys():
            if key in b.keys():
                total += (a[key] * b[key])
        return total
    
    # Takes a dictionary of TF-IDF values and returns magnitude
    def magnitude(self, v):
        total = 0
        for key in v.keys():
            total += (v[key] * v[key])
        return sqrt(total)
        

In [6]:
da = DocumentAnalyzer(doc.data)

In [7]:
da.compare(0, 2)

0.03349717081922913